In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from train_analysis_net import get_model, get_AFX_Chain, get_loaders, get_fx_list, Mel, get_datasets
import os
import librosa
import scipy.io
import audio_quality_estimation
import scipy.signal
from IPython.display import Audio

cm = 1/2.54

cuda:0


In [2]:
def dB20(x):
    return 20*np.log10(np.abs(x))

def RMS(x):
    return np.sqrt(np.mean(np.square(np.abs(x))))



device = 'cuda:0'
sr_hz = 44100

## Equalizer

In [3]:
encoder_type = "time_frequency"
afx_list =["peq", 'geq']
labels_list={"peq":"Parametric", "geq":'Graphic'}
sfx="peq"
loss_type="mel"
prefix=''
suffix=''

SFX_chain = get_fx_list(sfx, synthesis_ranges_dict_path="configs/synthesis_controls_ranges.yaml")


In [4]:
netAFX_list = {}

for afx in afx_list:
    AFX_Chain = get_AFX_Chain(model_str=afx)

    net=get_model(AFX_Chain=AFX_Chain, encoder_type=encoder_type).to(device).eval()
    experience_name = f"quality/{prefix}_SFX_{sfx}_AFX_{afx}_ENC_{encoder_type}_LOSS_mel_{suffix}"
    net.load_state_dict(
        torch.load(os.path.join('/tsi/data_doctorants/peladeau', 'trained_models', f'{experience_name}.pt'))
    )


    netAFX_list[afx]={
        'net':net,
        'fx':AFX_Chain,
        'label':labels_list[afx]
    }
l2 = torch.nn.MSELoss()

CQT kernels created, time used = 0.1101 seconds
CQT kernel width : 32768
CQT kernels created, time used = 0.0826 seconds
CQT kernel width : 32768


In [5]:
batch_size=1

_, _, test_dataset = get_datasets(sfx='', pathToMUSDB18='/home/ids/peladeau/Data/MUSDB18/')

print(len(test_dataset))

Training dataset length : 86
Validation dataset length : 14
Test dataset length : 50
50


In [6]:
x, y, q = test_dataset[10]
x=x.unsqueeze(0)
display(Audio(data=x[0,0].clone().cpu().numpy(), rate=44100))

In [7]:
styles = {
    "Graphic": '--',
    "Parametric" : '-.',
    "Ground Truth" : 'k-'}

In [8]:

compute=False
export= False
exports={}
if compute:

    x=x.to(device)


    outputs = {}

    with torch.no_grad():
        q = torch.rand(batch_size, SFX_chain.num_controls)


        imp = torch.zeros(batch_size, 1, sr_hz).to(device)
        for i in range(batch_size):
            imp[i,0,0]=1


        y = SFX_chain(x, q =q)
        
        outputs['Ground Truth'] = SFX_chain(imp, q =q).detach().cpu().numpy()
        
        exports['Input'] = x.clone().detach().cpu().numpy()
        exports['GT'] = y.clone().detach().cpu().numpy()

        for label, netAFX in netAFX_list.items():
            controller=netAFX['net']
            AFX_Chain=netAFX["fx"]
            qHat = controller(y)
            
            yHat = AFX_Chain(x, q =qHat)
            yHat = yHat/torch.sqrt(torch.mean(torch.square(yHat), axis=2, keepdim=True))
            exports[label] = yHat.detach().cpu().numpy()
            print(netAFX['label'])
            print(f'Mel metric: {round(torch.mean(Mel(y, yHat)).item(), 4)}')

            yHat = AFX_Chain(imp, q =qHat)
            yHat = yHat/torch.sqrt(torch.sum(torch.square(yHat), axis=2, keepdim=True))
            outputs[netAFX['label']] = yHat.detach().cpu().numpy()

max_val = 0
for label in exports.keys():
    exports[label] = exports[label]/RMS(exports[label])
    max_val = max(max_val, np.amax(np.abs(exports[label])))

for label in exports.keys():
    exports[label] = exports[label]/max_val

if export:
    for label, sig in exports.items():
        scipy.io.wavfile.write(filename=f'Exports/EQ/{label}.wav', rate=44100, data=sig)

    
plt.figure(figsize = (15*cm, 10*cm))
for label, sig in outputs.items():

    sig = sig[0,0,:]

    x_fft = np.fft.rfft(sig)
    f = np.fft.rfftfreq(n=len(sig), d = 1/44100)

    line = plt.plot(f, dB20(x_fft/RMS(sig)),styles[label], label = label)
plt.semilogx()
plt.xlim(20, 22050)
plt.ylim(-30, 30)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Gain (dB)")
plt.legend()
plt.tight_layout()
#plt.savefig("Images/EQ_Matching.svg")
plt.show()


    #plt.plot(q_np*30-15)
    #plt.plot(qHat_np*30-15)
    #plt.show()

NameError: name 'outputs' is not defined

<Figure size 590.551x393.701 with 0 Axes>